In [1]:
import requests
from selenium import webdriver
from webdriver_manager.utils import chrome_version
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
version = chrome_version()
url = 'http://chromedriver.storage.googleapis.com/LATEST_RELEASE_' + version
response = requests.get(url)
note_url = "https://note.com/search?context=user&q=%E3%82%A8%E3%83%B3%E3%82%B8%E3%83%8B%E3%82%A2&size=10%5C"

In [3]:
op = Options()
# --headlessだけではOSによって動かない、プロキシが弾かれる、
# CUI用の省略されたHTMLが帰ってくるなどの障害が出ます。
# 長いですが、これら6行あって最強かつどんな環境でも動きますので、必ず抜かさないようにしてください。
op.add_argument("--disable-gpu");
op.add_argument("--disable-extensions");
op.add_argument("--proxy-server='direct://'");
op.add_argument("--proxy-bypass-list=*");
op.add_argument("--start-maximized");
# op.add_argument("--headless");

# 要件定義

   空のurlと名前のリストを用意
    ↓
    初めのurlのリストの数を取得
    ↓
    スクロール前の状態のリストの数を取得
    ↓
    スクロールを実行*100回 : 0.4sごと
    ↓
    スクロール後のリストのカスを取得
    ↓
    スクロール前と後であとのほうが多かったらはじめに戻る　| 多かったら↓↓へ
    ↓↓
    urlと名前のリストを返す。

In [21]:
#driverの設定
def set_driver(op,note_url):
    driver = webdriver.Chrome(executable_path=ChromeDriverManager(response.text).install() , chrome_options=op)
    driver.get(note_url)
    return driver

In [22]:
def scroll(driver):
    judge=False
    while judge == False:
        try:
            user_url_list = []
            name_list = []
            note_len_before = driver.find_elements_by_class_name('o-searchResultUser__item')
            try:
                for i in range(100):
                    driver.execute_script('window.scrollTo(0 ,document.body.scrollHeight);')
                    time.sleep(0.3)
                note_len_after = driver.find_elements_by_class_name('o-searchResultUser__item')
                
                # 一番底につくまで処理を続ける
                if len(note_len_before) < len(note_len_after):
                    print(len(note_len_after))
                    print(name_list)

                # 一番底についたら要素を獲得する  
                elif len(note_len_before)==len(note_len_after):
                    for i in range(len(note_len_after)):
                        print(i)
                        userurl = note_len_after[i].find_elements_by_class_name('m-userListItem__link a-link')[0].get_attribute('href')
                        print(userurl)
                        user_url_list.append(userurl)
                        username = note_len_after[i].find_element_by_class_name('m-userListItem__nameLabel').text
                        name_list.append(username)
                        judge = True
                    
        except KeyboardInterrupt:
            judge = True
            
    return user_url_list,name_list,driver

In [23]:
def main():
    driver = set_driver(op,note_url)
    a,b,driver = scroll(driver)
    return a,b,driver

In [25]:
a,b,driver = main()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Driver [/Users/ryusei/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


330
[]


In [26]:
b

[]

In [10]:
df = pd.read_excel("noteのユーザー.xlsx")

In [11]:
df

,id,name,user_url,abstract,follow,follower
0,1,山本 一成🌤Ponanza,https://note.com/issei_y,名人に勝利した将棋プログラムPonanzaの作者です／東大卒／愛知学院大学特任准教授／東大客...,479,"45,504"
1,2,こんぴゅ,https://note.com/konpyu,web技術全般が専門のエンジニアです。ストレスなく理解できる技術記事を書いていきます,419,"43,469"
2,3,キャリアハック編集長 白石勝也,https://note.com/3lines,【テック × プレス3行まとめ】キャリアハック編集長／エンジニアHub 共同編集／AMBI編...,37,"41,921"
3,4,shu223,https://note.com/shu223,フリーランスiOSエンジニア兼サンフランシスコのFyusion社勤務。 著書に「iOS×BL...,71,"29,544"
4,5,中島聡,https://note.com/lifeisbeautiful,エンジニア、起業家、サイエンス・ライター。自分が創業者として米国に作った会社を２回売却（$5...,0,"20,444"
...,...,...,...,...,...,...
8224,8225,Tack Hershey,https://note.com/poko_suke,米州立大→輸出ビジネス→デイトレーダー→ソフトウェアエンジニア(フィンテック)→SGスタート...,47,3
8225,8226,Yoh Sano,https://note.com/yokosano,経歴：ソフトウェアエンジニア←機械学習エンジニア(コンピュータビジョン)←ブリッジエンジニア...,130,23
8226,8227,dottun,https://note.com/at23,電気系エンジニア/toeic705/今年40/子供3人/ 人生感謝しかないです。ツイッターは...,8,2
8227,8228,まき,https://note.com/makies,札幌でWeb系のエンジニアをしている。,46,26


In [10]:
import pandas as pd

In [11]:
import openpyxl
import xlwt

In [12]:
df.to_csv("user_note.csv")

NameError: name 'df' is not defined

In [13]:
dfdf = pd.read_csv("user_note.csv")

In [14]:
dfdf=dfdf.rename(columns={'Unnamed: 0':"user_number"})

In [15]:
#この操作は一回のみ
dfdf.user_number +=1

In [16]:
dfdf

,user_number,user_url,name
0,1,https://note.com/issei_y,山本 一成🌤Ponanza
1,2,https://note.com/konpyu,こんぴゅ
2,3,https://note.com/3lines,キャリアハック編集長 白石勝也
3,4,https://note.com/shu223,shu223
4,5,https://note.com/lifeisbeautiful,中島聡
...,...,...,...
8224,8225,https://note.com/poko_suke,Tack Hershey
8225,8226,https://note.com/yokosano,Yoh Sano
8226,8227,https://note.com/at23,dottun
8227,8228,https://note.com/makies,まき


# ここからフォロー数、フォロワー数、概要の取得

In [17]:
dfdf

,user_number,user_url,name
0,1,https://note.com/issei_y,山本 一成🌤Ponanza
1,2,https://note.com/konpyu,こんぴゅ
2,3,https://note.com/3lines,キャリアハック編集長 白石勝也
3,4,https://note.com/shu223,shu223
4,5,https://note.com/lifeisbeautiful,中島聡
...,...,...,...
8224,8225,https://note.com/poko_suke,Tack Hershey
8225,8226,https://note.com/yokosano,Yoh Sano
8226,8227,https://note.com/at23,dottun
8227,8228,https://note.com/makies,まき


In [18]:
dfdf1 = dfdf.iloc[:1000,:].user_url.reset_index()
dfdf2 = dfdf.iloc[1000:2000,:].user_url.reset_index()
dfdf3 = dfdf.iloc[2000:3000,:].user_url.reset_index()
dfdf4 = dfdf.iloc[3000:4000,:].user_url.reset_index()
dfdf5 = dfdf.iloc[4000:5000,:].user_url.reset_index()
dfdf6 = dfdf.iloc[5000:6000,:].user_url.reset_index()
dfdf7 = dfdf.iloc[6000:7000,:].user_url.reset_index()
dfdf8 = dfdf.iloc[7000:8000,:].user_url.reset_index()
dfdf9 = dfdf.iloc[8000:,:] .user_url.reset_index()

In [19]:
dfdf7[670:900]

,index,user_url
670,6670,https://note.com/g0e
671,6671,https://tencho.dev/
672,6672,https://note.com/pyokotan_miffi
673,6673,https://note.com/yasuhiro_kawasoe
674,6674,https://note.com/tomaron
...,...,...
895,6895,https://note.com/bomboclaat999
896,6896,https://note.com/yamashiro
897,6897,https://note.com/sendatu
898,6898,https://note.com/tsuchiya33333


In [26]:
dfdf6[990:-3]

,index,user_url
990,5990,https://note.com/elletech
991,5991,https://note.com/masaya_iwasaki
992,5992,https://note.com/joooi13
993,5993,https://note.com/faka
994,5994,https://note.com/yuko_ol
995,5995,https://note.com/koozaki
996,5996,https://note.com/ero_engineer


In [27]:
dfdf6[0:3]

,index,user_url
0,5000,https://note.com/fujiyamabitkey
1,5001,https://note.com/cktg
2,5002,https://note.com/you_managements


In [28]:
import chromedriver_binary

In [26]:
follow = []
follower = []
abst = []

In [27]:
c=[]
d=[]
e=[]

In [28]:
dfdf1

,user_number,user_url,name
0,1,https://note.com/issei_y,山本 一成🌤Ponanza
1,2,https://note.com/konpyu,こんぴゅ
2,3,https://note.com/3lines,キャリアハック編集長 白石勝也
3,4,https://note.com/shu223,shu223
4,5,https://note.com/lifeisbeautiful,中島聡
...,...,...,...
995,996,https://note.com/akirattii,akirattii
996,997,https://note.com/takuyasugarapple,拓弥 | 404DESIGNERS
997,998,https://note.com/showmiyu,しょうみゆ@WEBエンジニア
998,999,https://note.com/kusamao_tamb,Kusamao Abe


In [29]:
op = Options()
# --headlessだけではOSによって動かない、プロキシが弾かれる、
# CUI用の省略されたHTMLが帰ってくるなどの障害が出ます。
# 長いですが、これら6行あって最強かつどんな環境でも動きますので、必ず抜かさないようにしてください。
op.add_argument("no-sandbox")
op.add_argument("--disable-extensions")
op.add_argument("--headless")
op.add_argument('--disable-gpu')  
op.add_argument('--ignore-certificate-errors')
op.add_argument('--allow-running-insecure-content')
op.add_argument('--disable-web-security')
op.add_argument('--disable-desktop-notifications')
op.add_argument("--disable-extensions")
op.add_argument('--lang=ja')
op.add_argument('--blink-settings=imagesEnabled=false')
op.add_argument('--disable-dev-shm-usage')
op.add_argument('--proxy-server="direct://"')
op.add_argument('--proxy-bypass-list=*')    
op.add_argument('--start-maximized')

wait = WebDriverWait(driver, 300);
element=wait.until(EC.presence_of_element_located((By.CLASS_NAME,"fluid")));

In [30]:
def get_status(dfdf,op,follow,follower,abst):
    driver = webdriver.Chrome(executable_path=ChromeDriverManager(response.text).install() , chrome_options=op)
    driver.implicitly_wait(10) # 秒
    print("{}回動作を繰り返します".format(len(dfdf)))
    for i in range(len(dfdf)):
        #urlへのアクセス
        driver_path = dfdf.user_url[i]
        driver.get(driver_path)
        time.sleep(1)
        #htmlの解析
#         html = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        #概要の取得
        ab = soup.find("div", class_="m-profileBiography").get_text()
        abst.append(ab)
        #フォロー数の取得
        fol = soup.find_all("span",class_="m-profileStatus__followCount")[0].get_text()
        follow.append(fol)
        #フォロワー数の取得
        folw = soup.find_all("span",class_="m-profileStatus__followCount")[1].get_text()
        follower.append(folw)
        time.sleep(2)
        print(i+1)
    return abst, follow, follower

br.find_element_by_class_name("m-profileBiography").text
br.find_elements_by_class_name("m-profileStatus__followCount")[0].text
br.find_elements_by_class_name("m-profileStatus__followCount")[1].text

br.quit()

# 実際に使用するコード(詳細画面からの情報取得)

In [62]:
#dfdf1~dfdf8
def get_status_driver(dfdf,op,follow,follower,abst):
    print("{}回動作を繰り返します".format(len(dfdf)))
    for j in range(20):
        driver = webdriver.Chrome(chrome_options=op)
        driver.implicitly_wait(5) # 秒
        for i in range(50):
            #urlへのアクセス
#             j:0~19,i:0~49
            driver_path = dfdf.user_url[i+50*j]
            driver.get(driver_path)
            time.sleep(0.5)
            try:
                #概要の取得
                ab = driver.find_element_by_class_name("m-profileBiography").text
                #フォロー数の取得
                fol = driver.find_elements_by_class_name("m-profileStatus__followCount")[0].text
                #フォロワー数の取得
                folw = driver.find_elements_by_class_name("m-profileStatus__followCount")[1].text
                #獲得してから追加する
                follow.append(fol)
                abst.append(ab)
                follower.append(folw)
                time.sleep(0.3)
#             情報が足りなかった人がこれになる
            except:
                abst.append("error")
                follow.append("error")
                follower.append("error")
            print(i+50*j+1)
        driver.quit()
    return abst, follow, follower

In [32]:
c1,d1,e1=get_status_driver(dfdf1,op,follow=[],follower=[],abst=[])
c2,d2,e2=get_status_driver(dfdf2,op,follow=[],follower=[],abst=[])
c3,d3,e3=get_status_driver(dfdf3,op,follow=[],follower=[],abst=[])

1000回動作を繰り返します


/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


KeyboardInterrupt: 

driver = set_driver(op,driver_path)

In [20]:
def concat(a,b,c): #a,b,v=>list
    a,b,c = pd.DataFrame(a) , pd.DataFrame(b) ,pd.DataFrame(c)
    df = pd.concat([a,b,c],axis = 1)
    col = ["概要,”フォロー","フォロワー"]
    df.columns = col
    return df

In [27]:
df1 = pd.read_excel("3001_7000_kai.xlsx")

In [28]:
df1

,Unnamed: 0,abstract,follower,follow
0,0,サラリーマン。制御系のマイコン応用装置の開発エンジニアを経て、現在は事業企画でグローバルの生...,9,105
1,1,エンジニア×HR の世界にいるおじさん。日経コンピュータ IT業界新卒就職人気ランキングでは...,2,147
2,2,デジタルマーケティングエージェンシーのソフトウェアエンジニア。自然言語処理に傾倒。今は組織の...,0,43
3,3,ITエンジニア。社会人3年目25歳。 自死遺族だったり、性別がよくわからなかったり、傷つきや...,7,13
4,4,地方（京都）でテストエンジニアとして頑張っています。 学んだこと、日々感じることを記事にして...,5,7
...,...,...,...,...
3995,995,Y、Gなどで、web広告のエンジニア、プロデューサ、マネジメントなど経験してきました。先輩上...,29,112
3996,996,エンジニアです,2,6
3997,997,株式会社Palfit 代表取締役CEO｜フィットネス×食×D2Cの新規事業立ち上げをやってい...,11,19
3998,998,1985年9月9日生。日本大学藝術学部文芸学科卒。座右の銘は「たまに魔法を使う」。生業は W...,21,42


# データの統合

In [29]:
df_user = pd.read_csv("user_note.csv")
df1 = pd.read_csv("note_list1_3000.csv")
df2=pd.read_excel("3001_7000_kai.xlsx")
df3=pd.read_csv("note_list7001_8229.csv")

In [38]:
col= ["id","abstract","follower","follow"]

In [32]:
df2.columns=col_

df2.id+=1

In [41]:
df2

,id,abstract,follower,follow
0,1,サラリーマン。制御系のマイコン応用装置の開発エンジニアを経て、現在は事業企画でグローバルの生...,9,105
1,2,エンジニア×HR の世界にいるおじさん。日経コンピュータ IT業界新卒就職人気ランキングでは...,2,147
2,3,デジタルマーケティングエージェンシーのソフトウェアエンジニア。自然言語処理に傾倒。今は組織の...,0,43
3,4,ITエンジニア。社会人3年目25歳。 自死遺族だったり、性別がよくわからなかったり、傷つきや...,7,13
4,5,地方（京都）でテストエンジニアとして頑張っています。 学んだこと、日々感じることを記事にして...,5,7
...,...,...,...,...
3995,996,Y、Gなどで、web広告のエンジニア、プロデューサ、マネジメントなど経験してきました。先輩上...,29,112
3996,997,エンジニアです,2,6
3997,998,株式会社Palfit 代表取締役CEO｜フィットネス×食×D2Cの新規事業立ち上げをやってい...,11,19
3998,999,1985年9月9日生。日本大学藝術学部文芸学科卒。座右の銘は「たまに魔法を使う」。生業は W...,21,42


In [46]:
df2 =df2.reindex(["id","abstract","follow","follower"],axis="columns")

In [47]:
df1.columns = col

In [49]:
df1= df1.reindex(["id","abstract","follow","follower"],axis="columns")

In [50]:
df1

,id,abstract,follow,follower
0,0,名人に勝利した将棋プログラムPonanzaの作者です／東大卒／愛知学院大学特任准教授／東大客...,479,"45,504"
1,1,web技術全般が専門のエンジニアです。ストレスなく理解できる技術記事を書いていきます,419,"43,469"
2,2,【テック × プレス3行まとめ】キャリアハック編集長／エンジニアHub 共同編集／AMBI編...,37,"41,921"
3,3,フリーランスiOSエンジニア兼サンフランシスコのFyusion社勤務。 著書に「iOS×BL...,71,"29,544"
4,4,エンジニア、起業家、サイエンス・ライター。自分が創業者として米国に作った会社を２回売却（$5...,0,"20,444"
...,...,...,...,...
2995,995,地方国立大学院(非情報系)修了後、都内でエンジニアをしております。,3,0
2996,996,現在は外資系でマーケティングをしています。 学生時代は機械工学を学び、エンジニア職で役所に入...,3,3
2997,997,AI/IoT/仮想通貨などの技術情報について探りを入れてるエンジニア。筋量で投資に勝つ戦略を...,90,34
2998,998,都内在住 秋葉原で仕事するオタクエンジニア。 動画作ったり配信したりと器用貧乏を生かして生活...,12,2


In [52]:
df3.columns = col

In [53]:
df3= df3.reindex(["id","abstract","follow","follower"],axis="columns")

In [54]:
df3

,id,abstract,follow,follower
0,0,プログラマ歴30年。Webエンジニア歴20年。在宅ワーク歴15年。R50現役プログラマです。...,2,1
1,1,半導体エンジニア・・・かなぁ。漫画好きの引きこもり＆乗り鉄,2,1
2,2,詩人になれなかったソフトウェアエンジニア。理想と現実の狭間でもがきながら今日もどこかで生きて...,16,2
3,3,雑用エンジニア兼インターネットエンジニア。10年努めた大手企業を卒業して長野で好きなことやっ...,9,1
4,4,しがないエンジニア,62,5
...,...,...,...,...
1224,224,米州立大→輸出ビジネス→デイトレーダー→ソフトウェアエンジニア(フィンテック)→SGスタート...,47,3
1225,225,経歴：ソフトウェアエンジニア←機械学習エンジニア(コンピュータビジョン)←ブリッジエンジニア...,130,23
1226,226,電気系エンジニア/toeic705/今年40/子供3人/ 人生感謝しかないです。ツイッターは...,8,2
1227,227,札幌でWeb系のエンジニアをしている。,46,26


In [126]:
concat_data = pd.concat([df1,df2,df3])

In [127]:
concat_data

,id,abstract,follow,follower
0,0,名人に勝利した将棋プログラムPonanzaの作者です／東大卒／愛知学院大学特任准教授／東大客...,479,"45,504"
1,1,web技術全般が専門のエンジニアです。ストレスなく理解できる技術記事を書いていきます,419,"43,469"
2,2,【テック × プレス3行まとめ】キャリアハック編集長／エンジニアHub 共同編集／AMBI編...,37,"41,921"
3,3,フリーランスiOSエンジニア兼サンフランシスコのFyusion社勤務。 著書に「iOS×BL...,71,"29,544"
4,4,エンジニア、起業家、サイエンス・ライター。自分が創業者として米国に作った会社を２回売却（$5...,0,"20,444"
...,...,...,...,...
1224,224,米州立大→輸出ビジネス→デイトレーダー→ソフトウェアエンジニア(フィンテック)→SGスタート...,47,3
1225,225,経歴：ソフトウェアエンジニア←機械学習エンジニア(コンピュータビジョン)←ブリッジエンジニア...,130,23
1226,226,電気系エンジニア/toeic705/今年40/子供3人/ 人生感謝しかないです。ツイッターは...,8,2
1227,227,札幌でWeb系のエンジニアをしている。,46,26


In [128]:
import numpy as np

In [129]:
num = np.arange(1,8230)

In [130]:
len(num)

8229

In [131]:
num = num.reshape(8229,1)

In [132]:
num = pd.DataFrame(num)

In [133]:
concat_data

,id,abstract,follow,follower
0,0,名人に勝利した将棋プログラムPonanzaの作者です／東大卒／愛知学院大学特任准教授／東大客...,479,"45,504"
1,1,web技術全般が専門のエンジニアです。ストレスなく理解できる技術記事を書いていきます,419,"43,469"
2,2,【テック × プレス3行まとめ】キャリアハック編集長／エンジニアHub 共同編集／AMBI編...,37,"41,921"
3,3,フリーランスiOSエンジニア兼サンフランシスコのFyusion社勤務。 著書に「iOS×BL...,71,"29,544"
4,4,エンジニア、起業家、サイエンス・ライター。自分が創業者として米国に作った会社を２回売却（$5...,0,"20,444"
...,...,...,...,...
1224,224,米州立大→輸出ビジネス→デイトレーダー→ソフトウェアエンジニア(フィンテック)→SGスタート...,47,3
1225,225,経歴：ソフトウェアエンジニア←機械学習エンジニア(コンピュータビジョン)←ブリッジエンジニア...,130,23
1226,226,電気系エンジニア/toeic705/今年40/子供3人/ 人生感謝しかないです。ツイッターは...,8,2
1227,227,札幌でWeb系のエンジニアをしている。,46,26


In [134]:
num.columns=["idid"]

In [135]:
num

,idid
0,1
1,2
2,3
3,4
4,5
...,...
8224,8225
8225,8226
8226,8227
8227,8228


In [136]:
concat_data=concat_data.reset_index()

In [137]:
concat_data

,index,id,abstract,follow,follower
0,0,0,名人に勝利した将棋プログラムPonanzaの作者です／東大卒／愛知学院大学特任准教授／東大客...,479,"45,504"
1,1,1,web技術全般が専門のエンジニアです。ストレスなく理解できる技術記事を書いていきます,419,"43,469"
2,2,2,【テック × プレス3行まとめ】キャリアハック編集長／エンジニアHub 共同編集／AMBI編...,37,"41,921"
3,3,3,フリーランスiOSエンジニア兼サンフランシスコのFyusion社勤務。 著書に「iOS×BL...,71,"29,544"
4,4,4,エンジニア、起業家、サイエンス・ライター。自分が創業者として米国に作った会社を２回売却（$5...,0,"20,444"
...,...,...,...,...,...
8224,1224,224,米州立大→輸出ビジネス→デイトレーダー→ソフトウェアエンジニア(フィンテック)→SGスタート...,47,3
8225,1225,225,経歴：ソフトウェアエンジニア←機械学習エンジニア(コンピュータビジョン)←ブリッジエンジニア...,130,23
8226,1226,226,電気系エンジニア/toeic705/今年40/子供3人/ 人生感謝しかないです。ツイッターは...,8,2
8227,1227,227,札幌でWeb系のエンジニアをしている。,46,26


In [139]:
concat_semi = pd.concat([num,concat_data],axis=1)

In [140]:
concat_semi

,idid,index,id,abstract,follow,follower
0,1,0,0,名人に勝利した将棋プログラムPonanzaの作者です／東大卒／愛知学院大学特任准教授／東大客...,479,"45,504"
1,2,1,1,web技術全般が専門のエンジニアです。ストレスなく理解できる技術記事を書いていきます,419,"43,469"
2,3,2,2,【テック × プレス3行まとめ】キャリアハック編集長／エンジニアHub 共同編集／AMBI編...,37,"41,921"
3,4,3,3,フリーランスiOSエンジニア兼サンフランシスコのFyusion社勤務。 著書に「iOS×BL...,71,"29,544"
4,5,4,4,エンジニア、起業家、サイエンス・ライター。自分が創業者として米国に作った会社を２回売却（$5...,0,"20,444"
...,...,...,...,...,...,...
8224,8225,1224,224,米州立大→輸出ビジネス→デイトレーダー→ソフトウェアエンジニア(フィンテック)→SGスタート...,47,3
8225,8226,1225,225,経歴：ソフトウェアエンジニア←機械学習エンジニア(コンピュータビジョン)←ブリッジエンジニア...,130,23
8226,8227,1226,226,電気系エンジニア/toeic705/今年40/子供3人/ 人生感謝しかないです。ツイッターは...,8,2
8227,8228,1227,227,札幌でWeb系のエンジニアをしている。,46,26


In [141]:
#不要なデータの消去
concat_semi = concat_semi.drop("id",axis=1)
concat_semi = concat_semi.drop("index",axis=1)

In [152]:
#列名の書き換え
concat_semi = concat_semi.rename(columns={"idid":"id"})

In [153]:
concat_semi

,id,abstract,follow,follower
0,1,名人に勝利した将棋プログラムPonanzaの作者です／東大卒／愛知学院大学特任准教授／東大客...,479,"45,504"
1,2,web技術全般が専門のエンジニアです。ストレスなく理解できる技術記事を書いていきます,419,"43,469"
2,3,【テック × プレス3行まとめ】キャリアハック編集長／エンジニアHub 共同編集／AMBI編...,37,"41,921"
3,4,フリーランスiOSエンジニア兼サンフランシスコのFyusion社勤務。 著書に「iOS×BL...,71,"29,544"
4,5,エンジニア、起業家、サイエンス・ライター。自分が創業者として米国に作った会社を２回売却（$5...,0,"20,444"
...,...,...,...,...
8224,8225,米州立大→輸出ビジネス→デイトレーダー→ソフトウェアエンジニア(フィンテック)→SGスタート...,47,3
8225,8226,経歴：ソフトウェアエンジニア←機械学習エンジニア(コンピュータビジョン)←ブリッジエンジニア...,130,23
8226,8227,電気系エンジニア/toeic705/今年40/子供3人/ 人生感謝しかないです。ツイッターは...,8,2
8227,8228,札幌でWeb系のエンジニアをしている。,46,26


In [148]:
df_user = df_user.iloc[:,1:]

In [149]:
df_user

,user_url,name
0,https://note.com/issei_y,山本 一成🌤Ponanza
1,https://note.com/konpyu,こんぴゅ
2,https://note.com/3lines,キャリアハック編集長 白石勝也
3,https://note.com/shu223,shu223
4,https://note.com/lifeisbeautiful,中島聡
...,...,...
8224,https://note.com/poko_suke,Tack Hershey
8225,https://note.com/yokosano,Yoh Sano
8226,https://note.com/at23,dottun
8227,https://note.com/makies,まき


In [160]:
df_final = pd.concat([concat_semi,df_user],axis=1)

In [161]:
df_final.columns

Index(['id', 'abstract', 'follow', 'follower', 'user_url', 'name'], dtype='object')

In [162]:
col_change=["id","name","user_url","abstract","follow","follower"]

In [164]:
df_final = df_final.reindex(col_change,axis="columns")

In [165]:
df_final

,id,name,user_url,abstract,follow,follower
0,1,山本 一成🌤Ponanza,https://note.com/issei_y,名人に勝利した将棋プログラムPonanzaの作者です／東大卒／愛知学院大学特任准教授／東大客...,479,"45,504"
1,2,こんぴゅ,https://note.com/konpyu,web技術全般が専門のエンジニアです。ストレスなく理解できる技術記事を書いていきます,419,"43,469"
2,3,キャリアハック編集長 白石勝也,https://note.com/3lines,【テック × プレス3行まとめ】キャリアハック編集長／エンジニアHub 共同編集／AMBI編...,37,"41,921"
3,4,shu223,https://note.com/shu223,フリーランスiOSエンジニア兼サンフランシスコのFyusion社勤務。 著書に「iOS×BL...,71,"29,544"
4,5,中島聡,https://note.com/lifeisbeautiful,エンジニア、起業家、サイエンス・ライター。自分が創業者として米国に作った会社を２回売却（$5...,0,"20,444"
...,...,...,...,...,...,...
8224,8225,Tack Hershey,https://note.com/poko_suke,米州立大→輸出ビジネス→デイトレーダー→ソフトウェアエンジニア(フィンテック)→SGスタート...,47,3
8225,8226,Yoh Sano,https://note.com/yokosano,経歴：ソフトウェアエンジニア←機械学習エンジニア(コンピュータビジョン)←ブリッジエンジニア...,130,23
8226,8227,dottun,https://note.com/at23,電気系エンジニア/toeic705/今年40/子供3人/ 人生感謝しかないです。ツイッターは...,8,2
8227,8228,まき,https://note.com/makies,札幌でWeb系のエンジニアをしている。,46,26


In [169]:
df_final.to_excel("complete_notedata.xlsx",engine="xlsxwriter")
df_final.to_csv("complete_notedata.csv")

In [170]:
ls

3001_7000.xlsx          kai.csv                 user_note
3001_7000_kai.xlsx      note.ipynb              user_note.csv
complete_notedata.csv   note_list1_3000.csv     ~$3001_7000_kai.xlsx
complete_notedata.xlsx  note_list7001_8229.csv
